In [3]:
! export JAX_PLATFORMS=cpu

/gpfs/cfel/group/cmi/common/psi4/psi4conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


# Hyperfine solutions for H2S molecule (spin-rotation interaction)



In [4]:
import h5py
import jax
import numpy as np
from jax import config
from jax import numpy as jnp

from rovib.symtop import threej_wang
from rovib.cartens import CART_IND

config.update("jax_enable_x64", True)

Read vibrational matrix elements of spin-rotation tensors and compute their matrix elements in rovibrational basis

In [5]:
def tensor_rovib_me(
    rank: int, j1: int, j2: int, vib_me, pmax: int = 20, linear: bool = False, tol=1e-12
):
    """Computes rovibrational matrix elements of a Cartesian tensor operator.

    This function calculates the rovibrational matrix elements using wavefunctions stored in 
    separate files for different values of the J quantum number (see `h2s_rovib.ipynb`). 
    The computation is performed for the specified bra (`j1`) and ket (`j2`) values of the J
    quantum number.
    """

    # determine the order of Cartesian indices in the Cartesian-to-spherical tensor
    #   transformation matrix (in cartens.CART_IND and threej_wang)
    cart_ind = [["xyz".index(x) for x in elem] for elem in CART_IND[rank]]

    # reshape vibrational matrix elements such that the order of Cartesian indices
    #   correspond to the order in threej_wang output
    vib_me_ = np.moveaxis([vib_me[:, :, i, j] for (i, j) in cart_ind], 0, -1)

    # rotational matrix elements, rot_me[omega].shape = (nbas_rot, nbas_rot, ncart)
    jktau_list1, jktau_list2, rot_me = threej_wang(rank, j1, j2, linear=linear)

    h5_1 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j1}.h5", "r")
    h5_2 = h5py.File(f"h2s_coefficients_pmax{pmax}_j{j2}.h5", "r")

    res = {}

    for sym1 in h5_1["energies"].keys():
        enr1 = h5_1["energies"][sym1][:]
        coefs1 = h5_1["coefficients"][sym1][:]
        vind1 = h5_1["vib-indices"][sym1][:]
        rind1 = h5_1["rot-indices"][sym1][:]
        qua1 = np.array(
            [elem[0].decode("utf-8").split(",") for elem in h5_1["quanta"][sym1][:]]
        )
        for sym2 in h5_2["energies"].keys():
            enr2 = h5_2["energies"][sym2][:]
            coefs2 = h5_2["coefficients"][sym2][:]
            vind2 = h5_2["vib-indices"][sym2][:]
            rind2 = h5_2["rot-indices"][sym2][:]
            qua2 = np.array(
                [elem[0].decode("utf-8").split(",") for elem in h5_2["quanta"][sym2][:]]
            )

            me = []
            for omega in range(rank + 1):
                me_ = np.einsum(
                    "ijc,ijc->ij",
                    vib_me_[np.ix_(vind1, vind2)],
                    rot_me[omega][np.ix_(rind1, rind2)],
                    optimize="optimal",
                )
                me.append(
                    np.einsum(
                        "ik,ij,jl->kl", np.conj(coefs1), me_, coefs2, optimize="optimal"
                    )
                )

            me = np.array(me)
            me[np.abs(me) < tol] = 0
            if np.count_nonzero(me) > 0:
                res[(sym1, sym2)] = np.array(me)

    return res

In [16]:
pmax = 20

# read vibrational matrix elements of spin-rotation tensors

with h5py.File(f"h2s_vibme_pmax{pmax}.h5", "r") as h5:
    sr_h1_vib = h5["spin-rotation"]["h1"][:]
    sr_h2_vib = h5["spin-rotation"]["h2"][:]

# compute rovibrational matrix elements for different pairs of J quanta

rank = 2
max_j = 60
min_j = 60
me_sr1 = {}
for j1 in range(min_j, max_j + 1):
    for j2 in range(min_j, max_j + 1):
        me = tensor_rovib_me(rank, j1, j2, sr_h1_vib)
        print(j1, j2, list(me.keys()), [elem.shape for elem in me.values()])
        me_sr1[(j1, j2)] = me

60 60 [('A1', 'A1'), ('A1', 'B2'), ('A2', 'A2'), ('A2', 'B1'), ('B1', 'A2'), ('B1', 'B1'), ('B2', 'A1'), ('B2', 'B2')] [(3, 4869, 4869), (3, 4869, 4848), (3, 4653, 4653), (3, 4653, 4648), (3, 4648, 4653), (3, 4648, 4648), (3, 4848, 4869), (3, 4848, 4848)]


In [15]:
j1, j2 = (0,1)
sym1, sym2 = ('A1', 'B2')
for omega, me in enumerate(me_sr1[(j1,j2)][(sym1, sym2)]):
    print(omega)
    print(me)

0
[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]
1
[[-7.81720144e-01+0.j -2.32290210e-04+0.j  1.82633560e-04+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j]
 [-2.32378453e-04+0.j  7.81720008e-01+0.j -3.66227922e-04+0.j ...
   0.00000000e+00+0.j  0.00000000e+00+0.j  2.48821529e-12+0.j]
 [ 1.82433488e-04+0.j  3.66502418e-04+0.j  7.81719754e-01+0.j ...
  -1.02879483e-12+0.j  2.01686733e-12+0.j  2.59723749e-11+0.j]
 ...
 [ 0.00000000e+00+0.j  1.53517628e-11+0.j  4.07084135e-11+0.j ...
  -1.65814863e-02+0.j  1.41688069e-03+0.j -6.72200119e-05+0.j]
 [ 0.00000000e+00+0.j -1.08197883e-12+0.j -1.78160413e-12+0.j ...
  -7.81507268e-01+0.j -5.71114566e-03+0.j -5.72321226e-06+0.j]
 [ 0.00000000e+00+0.j  0.00000000e+00+0.j  2.0